In [4]:
import sys
sys.path.append("../")
from PIL import Image
import numpy as np
import os
import torch
from pathlib import Path
import rasterio
from rasterio.mask import mask
from shapely.geometry import box
import json
from torchvision import datasets, transforms
import random
import pycrs
from tqdm import tqdm
import geopandas as gpd
import gc
from split_tiff import Split

split=Split()
torch.cuda.empty_cache()
gc.collect()

data_path= "/mnt/9TB/koubaa/23_06_26_09"
onlyfiles = [f for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f))]

In [5]:
random.shuffle(onlyfiles)
print(len(onlyfiles))


5308


In [ ]:
means=torch.zeros(13,device='cuda:0')
std=torch.zeros(13,device='cuda:0')
for i in range(53):
    onlyfiles1=onlyfiles[100*i:100*(i+1)]
    images= split.ExtractImages_imagesonly(data_path,onlyfiles1)
    to_tensor=  transforms.ToTensor()
    device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'
    images= np.concatenate(images,1)
    images= to_tensor(images)
    images=images.to(device).detach()
    a = torch.cuda.memory_allocated(0)
    r= torch.cuda.memory_reserved(0)
    print(f'mmeansemory allocated:{a/(1024.0*1024.0*1024.0)}/ memory reserved:{r/(1024.0*1024.0*1024.0)}')
    means = means-means/(i+1)+torch.mean(images,2).squeeze()/(i+1)
    std = std-std/(i+1)+torch.std(images,2).squeeze()/(i+1)
    print(means)
    print(std)




Extracting images:   0%|                                                                                                                                                                   | 0/100 [00:00<?, ?it/s]Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


Extracting images: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [04:20<00:00,  2.61s/it]


mmeansemory allocated:6.974609375/ memory reserved:6.974609375


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [ ]:
data = [f.replace(".tiff","").split("_") for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f))]
rastercenters= np.array([np.array([float(d[2]),float(d[1])]) for d in data])
rastercenters=torch.from_numpy(rastercenters)
rastercenters=rastercenters.to('cuda:0')
meancoor =torch.mean(rastercenters,0)
stdcoor =torch.std(rastercenters,0)
print (meancoor)
print (stdcoor)


tensor([11.0551, 50.8178], device='cuda:0', dtype=torch.float64)
tensor([1.9841, 1.8167], device='cuda:0', dtype=torch.float64)
